In [1]:
# we'll take data from http://yann.lecun.com/exdb/mnist/ for digits

import tensorflow as tf

In [2]:
# basic idea of the NN
#
# repeat milions of times the two steps bellow:
#
# step 1: feed froward
#
# we get input image -> send it to input layer -> weights 
#   -> hidden layer 1 -> weights 
#   -> hidden layer 2 -> weights 
#   -> output 
#
# step 2: backpropagation
#
# compare output to intended outpuntt (we have indended from the data) -> cost function (corss entropy)
# cross entropy wiki: https://en.wikipedia.org/wiki/Cross_entropy
#
# optimization function -> attempts to mimzie cost function by manipulating the weights
# AdamOptimizer, SGD (gradient descnet) etc
# 
# feed forward + backpropagation = epoc (one iteration)

In [3]:
# we already have the data available in the examples file

from tensorflow.examples.tutorials.mnist import input_data

# one_hot = True means we represent the data in a vector way, see below
mnist = input_data.read_data_sets('/tmp/data', one_hot=True)

# 10 classes, from 0 to 9
# 0 = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] because one_hot=true in read_data_sets
# 1 = [0, 1, ....] etc

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [4]:
# we declare the number of newurons for each layer to be used 
# in configuration for tensureflow

n_nodes_hl1 = 500
n_nodes_hl2 = 620
n_nodes_hl3 = 300

n_classes = 10

# batch size = how many images we will send once before we backpropagate
# how many photos we send before we change weights in the neural network
batch_size = 100

In [5]:
# we declare inputs for the image. In tensorflow we declare the inputs
# as these placeholder which will be treated as input 'gates' of sorts
# when we run the program

# the height x width of the image, so the number of pixels we are sending 
x = tf.placeholder(tf.float32, [None, 28*28])

# y is what will be placing as expected values
y = tf.placeholder(tf.float32)

In [6]:
# each layer in the network will have one weight and one baias for each 'neuron'
# each neuron will receive all the inputs. 
# 
# this means that for each input value, we will run all the wieghts and add all the baiases?
def neural_network_layer_initialisation(input_count, output_count):
    layer_configuration = {
        'weigths': tf.Variable(
            # gets a random matrix of values, with inputcount rows and outputcount columns
            tf.random_normal([
                input_count, output_count
            ])
        ),
        'biases': tf.Variable(
            # gets a random row of values, of output_count
            tf.random_normal(
                [output_count]
            )
        )
    }
    return layer_configuration
    
# cool reason for biases. Because we might have 0 input, and 0 * weights = 0 all the time. 
# with baias, we might end up with neurons being triggered regardless of 0. 
# y = w * x + b (wieghts multiplied by input, adding bias afterwards) => output

In [7]:
def neural_network_model(data): 
    # first layer gets the image, so 784 pixel values and returns one value out of 
    # each neuron of the first layer
    hidden_1_layer = neural_network_layer_initialisation(784, n_nodes_hl1)
    # second layer gets the input from first layer and outputs a set of values
    # equal with the number of neurons on the second layer
    hidden_2_layer = neural_network_layer_initialisation(n_nodes_hl1, n_nodes_hl2)

    hidden_3_layer = neural_network_layer_initialisation(n_nodes_hl2, n_nodes_hl3)
    
    # output get the values from the third layer and outputs one number for each n_classes
    # we have. We will train the network and force it to return the one_hot=True representation
    # we discussed above, so for 0, we would expect the output to be [1, 0, 0 ...]
    output_layer = neural_network_layer_initialisation(n_nodes_hl3, n_classes)    
    
    
    # we now describe in therms that tensurflow understand the way the layers 
    # propagate data, in terms of matrix computations
    l1 = tf.add(
        tf.matmul(data, hidden_1_layer['weigths']), hidden_1_layer['biases']
    )
    ## todo read about
    l1 = tf.nn.relu(l1)
    
    
    # layer two takes layer 1 output and does its thing
    l2 = tf.add(
        tf.matmul(l1, hidden_2_layer['weigths']), hidden_2_layer['biases']
    )
    l2 = tf.nn.relu(l2)
    
    
    # layer two takes layer 2 output and does its thing
    l3 = tf.add(
        tf.matmul(l2, hidden_3_layer['weigths']), hidden_3_layer['biases']
    )
    l3 = tf.nn.relu(l3)
    
    print output_layer
    # output layer takes layer 3 output and does its thing
    output = tf.matmul(l3, output_layer['weigths']) + output_layer['biases']
    
    return output
    

In [16]:
def train_neural_netowrk(x, y, hm_epochs):
    
    # the final step of the configuration, we specify how the training session would behave
    prediction = neural_network_model(x)
    
    cost = tf.reduce_mean( 
        tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) 
    )
    
    # learning_rate = 0.001 is the default learning rate for this optimizer
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    correct = tf.equal(tf.argmax(prediction,1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    
    # we begin the training
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        for epoch in range(1, hm_epochs):
            
            epoch_loss = 0
            
            for _ in range(int(mnist.train.num_examples/batch_size)):
                # it so happens that the minst data set gives us batches of images
                # via this helper function so we can focus on JUST the AI implementation
                e_x, e_y = mnist.train.next_batch(batch_size)
                
                a, c = sess.run([optimizer, cost], feed_dict = { x:e_x, y:e_y })
                
                epoch_loss += c
            
            print('Epoc', epoch, 'complete out of', hm_epochs, 'loss:', epoch_loss)
            print('Accuracy:', accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))
        
        
        print('Accuracy:', accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))
        
        
                

In [17]:
train_neural_netowrk(x, y, 20)

{'weigths': <tf.Variable 'Variable_38:0' shape=(300, 10) dtype=float32_ref>, 'biases': <tf.Variable 'Variable_39:0' shape=(10,) dtype=float32_ref>}
('Epoc', 1, 'complete out of', 20, 'loss:', 1326739.6838684082)
('Accuracy:', 0.90100002)


KeyboardInterrupt: 